In [44]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import pickle
from edmAnalyzer import binCalculator
from edmAnalyzer import binCutter
import pandas as pd
import numpy as np

In [45]:
with open(r"C:\ACME_analysis\Noise0010.2114\Run Results\data\binpara_offsettrace46_bincut_frac15_blockpara_simple2_blockcut_blc1_config_0\Binary Results\binresult_0010.2114.0000.0000.pkl", "rb") as f:
    a = pickle.load(f)
with open(r"C:\ACME_analysis\Noise0010.2114\Run Results\data\binpara_offsettrace46_bincut_frac15_blockpara_simple2_blockcut_blc1_config_0\Binary Results\bincutresult_0010.2114.0000.0000.pkl", "rb") as f:
    c = pickle.load(f)

In [46]:
def populate_df_from_bin(df, binresult, bincutresult):
    def ensure_columns(df, columns):
        for col in columns:
            if col not in df.columns:
                df[col] = 0

    def parse_name(name):
        parts = name.split('.')
        run = int(parts[0])
        sequence = int(parts[1])
        block = int(parts[2])
        return run, sequence, block

    def weighted_average(values, errors_squared, axis):
        weights = np.where(errors_squared > 0, 1.0 / errors_squared, 0.0)
        weighted_sum = np.sum(values * weights, axis=axis)
        total_weight = np.sum(weights, axis=axis)
        with np.errstate(divide='ignore', invalid='ignore'):
            average = np.where(total_weight > 0, weighted_sum / total_weight, 0)
        return average

    for a, c in zip(binresult, bincutresult):
        run, sequence, block = parse_name(a.name)
        idx_slice = slice(c.grand_left, c.grand_right + 1)
        num_traces = a.A.shape[0]

        for trace_idx in range(num_traces):
            row_idx = (df['run'] == run) & (df['sequence'] == sequence) & (df['block'] == block) & (df['trace'] == trace_idx)
            if not row_idx.any():
                continue

            for sipm in range(8):
                asym_col = f'Asymmetry_sipm{sipm}'
                fx_col = f'Fx_sipm{sipm}'
                fy_col = f'Fy_sipm{sipm}'
                num_col = f'Number_sipm{sipm}'
                ensure_columns(df, [asym_col, fx_col, fy_col, num_col])

                values = a.A[trace_idx, :, sipm, idx_slice]
                errors_squared = a.dA2_from_photon[trace_idx, :, sipm, idx_slice]
                avg = weighted_average(values, errors_squared, axis=(0, 1))
                df.loc[row_idx, asym_col] = avg

                fx_sum = np.sum(a.Fx[trace_idx, :, sipm, idx_slice])
                df.loc[row_idx, fx_col] = fx_sum

                fy_sum = np.sum(a.Fy[trace_idx, :, sipm, idx_slice])
                df.loc[row_idx, fy_col] = fy_sum

                n_sum = np.sum(a.N[trace_idx, :, sipm, idx_slice])
                df.loc[row_idx, num_col] = n_sum

            ensure_columns(df, ['Asymmetry_sipmavg', 'Fx_sipmavg', 'Fy_sipmavg', 'Number_sipmavg'])

            values = a.A[trace_idx, :, :, idx_slice]
            errors_squared = a.dA2_from_photon[trace_idx, :, :, idx_slice]
            avg_all = weighted_average(values, errors_squared, axis=(0, 1, 2))
            df.loc[row_idx, 'Asymmetry_sipmavg'] = avg_all

            fx_sum_all = np.sum(a.Fx[trace_idx, :, :, idx_slice])
            df.loc[row_idx, 'Fx_sipmavg'] = fx_sum_all

            fy_sum_all = np.sum(a.Fy[trace_idx, :, :, idx_slice])
            df.loc[row_idx, 'Fy_sipmavg'] = fy_sum_all

            n_sum_all = np.sum(a.N[trace_idx, :, :, idx_slice])
            df.loc[row_idx, 'Number_sipmavg'] = n_sum_all

    return df

In [47]:
df = pd.read_csv(r"C:\ACME_analysis\Noise0010.2114\Run Results\data\binpara_offsettrace46_bincut_frac15_blockpara_simple2_blockcut_blc1_config_0\Binary Results\bin_df_0010.2114.csv")

In [48]:
populate_df_from_bin(df, [a] , [c])

,run,sequence,block,trace,start year,start month,start day,start hour,start minute,start second,...,Fy_sipm6,Number_sipm6,Asymmetry_sipm7,Fx_sipm7,Fy_sipm7,Number_sipm7,Asymmetry_sipmavg,Fx_sipmavg,Fy_sipmavg,Number_sipmavg
0,10,2114,0,0,2025,4,25,17,27,55.863,...,4.315484e+06,9.361977e+06,0.084438,4.787960e+06,4.042350e+06,8.830309e+06,0.084035,3.731737e+07,3.153216e+07,6.884953e+07
1,10,2114,0,1,2025,4,25,17,27,56.822,...,4.192010e+06,9.125425e+06,0.087371,4.686277e+06,3.933231e+06,8.619508e+06,0.086606,3.648452e+07,3.066929e+07,6.715381e+07
2,10,2114,0,2,2025,4,25,17,27,57.632,...,4.182195e+06,9.099773e+06,0.086345,4.662421e+06,3.921258e+06,8.583679e+06,0.086626,3.639664e+07,3.059396e+07,6.699061e+07
3,10,2114,0,3,2025,4,25,17,27,58.454,...,4.184200e+06,9.086181e+06,0.084484,4.644727e+06,3.920962e+06,8.565689e+06,0.084191,3.626772e+07,3.063557e+07,6.690329e+07
4,10,2114,0,4,2025,4,25,17,27,59.507,...,4.833078e+06,9.133532e+06,-0.051855,4.071590e+06,4.516955e+06,8.588545e+06,-0.052714,3.187238e+07,3.541903e+07,6.729141e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,10,2114,63,11,2025,4,25,17,47,49.135,...,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
1020,10,2114,63,12,2025,4,25,17,47,50.180,...,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
1021,10,2114,63,13,2025,4,25,17,47,51.003,...,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
1022,10,2114,63,14,2025,4,25,17,47,51.814,...,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00


In [60]:
import pandas as pd
import itertools


def logging_channels_parity_transform(df: pd.DataFrame, exemption_list: list) -> pd.DataFrame:
    """
    Generate a new DataFrame with parity-transformed columns based on N, E, theta, B.

    Parameters:
    df : pd.DataFrame
        The original dataframe with columns: run, sequence, block, trace, N, E, theta, B, and others.
    exemption_list : list
        List of column names to be excluded from parity transformation.

    Returns:
    pd.DataFrame
        New dataframe grouped by (run, sequence, block) with parity-transformed columns.
    """
    df = df.copy()
    df['Q'] = df['theta']

    transform_columns = [col for col in df.columns 
                         if col not in exemption_list and col not in ['run', 'sequence', 'block', 'trace', 'N', 'E', 'theta', 'B']]

    group_keys = ['run', 'sequence', 'block']
    grouped = df.groupby(group_keys)

    new_records = []

    labels = ['N', 'E', 'Q', 'B']
    subsets = []
    for r in range(0, len(labels) + 1):
        for comb in itertools.combinations(labels, r):
            subsets.append(comb)

    for name, group in grouped:
        new_row = {k: v for k, v in zip(group_keys, name)}
        new_row['N'] = group['N'].mean()
        new_row['E'] = group['E'].mean()
        new_row['Q'] = group['Q'].mean()
        new_row['B'] = group['B'].mean()

        for col in transform_columns:
            for subset in subsets:
                if len(subset) == 0:
                    new_col_name = f"{col}_nr"
                    label_product = pd.Series(1, index=group.index)
                else:
                    label_product = pd.Series(1, index=group.index)
                    for label in subset:
                        label_product *= group[label]
                    new_col_name = f"{col}_{''.join(subset)}"

                values = group[col] * label_product
                avg_value = values.mean()

                new_row[new_col_name] = avg_value

        new_records.append(new_row)

    result_df = pd.DataFrame(new_records)
    return result_df

In [71]:
df = pd.read_csv(r"C:\ACME_analysis\Noise0010.2114\purged_aggregated_header.csv")

In [79]:
new_df = logging_channels_parity_transform(df, exemption_list = [])

In [78]:
import pandas as pd
import itertools
import numpy as np
from joblib import Parallel, delayed

def logging_channels_parity_transform(df: pd.DataFrame, exemption_list: list, nr_only_list: list = None) -> pd.DataFrame:
    """
    Generate a new DataFrame with parity-transformed columns based on N, E, theta, B.

    Parameters:
    df : pd.DataFrame
        The original dataframe with columns: run, sequence, block, trace, N, E, theta, B, and others.
    exemption_list : list
        List of column names to be excluded from parity transformation.
    nr_only_list : list, optional
        List of column names for which only non-reversing (nr) averaging is performed.

    Returns:
    pd.DataFrame
        New dataframe grouped by (run, sequence, block) with parity-transformed columns.
    """
    if nr_only_list is None:
        nr_only_list = []

    df = df.copy()
    df['Q'] = df['theta']

    transform_columns = [col for col in df.columns 
                         if col not in exemption_list and col not in ['run', 'sequence', 'block', 'trace', 'N', 'E', 'theta', 'B','Q']]

    group_keys = ['run', 'sequence', 'block']
    grouped = df.groupby(group_keys, sort=False)

    labels = ['N', 'E', 'Q', 'B']
    subsets = []
    subset_names = []
    for r in range(0, len(labels) + 1):
        for comb in itertools.combinations(labels, r):
            subsets.append(list(comb))
            if len(comb) == 0:
                subset_names.append('nr')
            else:
                subset_names.append(''.join(comb))

    def process_group(name_group):
        name, group = name_group
        base_row = {k: v for k, v in zip(group_keys, name)}
        base_row.update({k: group[k].mean() for k in labels})

        group_label_products = {}
        for subset, subset_name in zip(subsets, subset_names):
            if subset_name == 'nr':
                group_label_products[subset_name] = np.ones(len(group))
            else:
                group_label_products[subset_name] = group[subset].prod(axis=1)

        for col in transform_columns:
            if col in nr_only_list:
                weighted_avg = (group[col]).mean()
                base_row[f"{col}_nr"] = weighted_avg
            else:
                for subset_name in subset_names:
                    weighted_avg = (group[col] * group_label_products[subset_name]).mean()
                    base_row[f"{col}_{subset_name}"] = weighted_avg

        return base_row

    records = Parallel(n_jobs=-1)(delayed(process_group)(item) for item in grouped)

    result_df = pd.DataFrame.from_records(records)
    return result_df


In [81]:
new_df['XY_power_sum_N']

0    -0.000315
1     0.000915
2     0.000267
3     0.000478
4     0.001049
        ...   
59    0.000359
60    0.000776
61   -0.000933
62   -0.001004
63   -0.001565
Name: XY_power_sum_N, Length: 64, dtype: float64

In [ ]:
new_df['XY_power_sum_N']

In [84]:
new_df['XY_power_imbalance_N']

0    -0.001887
1    -0.001720
2    -0.001514
3    -0.001294
4    -0.001560
        ...   
59    0.001305
60    0.001260
61    0.001122
62    0.001216
63    0.001447
Name: XY_power_imbalance_N, Length: 64, dtype: float64

In [82]:
new_df['XY_power_imbalance_N']

0    -0.001887
1    -0.001720
2    -0.001514
3    -0.001294
4    -0.001560
        ...   
59    0.001305
60    0.001260
61    0.001122
62    0.001216
63    0.001447
Name: XY_power_imbalance_N, Length: 64, dtype: float64